In [1]:
import numpy as np
import pandas as pd
from functools import reduce

import warnings
warnings.filterwarnings("ignore")

## **Day 1**

In [103]:
a = pd.read_csv("inputs20/1", header=None).to_numpy().reshape(-1)
n = len(a)

for i in range(n):
    for j in range(i, n):
        if a[i] + a[j] == 2020:
            t1 = a[i] * a[j]
        for k in range(j, n):
            if a[i] + a[j] + a[k] == 2020:
                t2 = a[i] * a[j] * a[k]

print(f"Task 1: {t1}, Task 2: {t2}")

Task 1: 482811, Task 2: 193171814


**Alternative**: using the `in` keyword and sets to do Θ(1) (O(n)) lookup of the 2nd (3rd element), reducing complexity by `N`. [Source](https://www.reddit.com/r/adventofcode/comments/k4e4lm/comment/ge8s76t/?utm_source=share&utm_medium=web2x&context=3)

In [107]:
b = set(a)

for i in range(n):
    x = 2020 - a[i]
    if x in b:
        t1 = x * a[i]
    for j in range(n):
        xx = 2020 - a[i] - a[j]
        if xx in b:
            t1 = xx * a[i] * a[j]

print(f"Task 1: {t1}, Task 2: {t2}")

Task 1: 193171814, Task 2: 193171814


## **Day 2**

In [69]:
df = pd.read_csv("inputs20/2", header=None, sep=": | |-", engine="python")

iv1 = lambda r: r[0] <= r[3].count(r[2]) <= r[1]
iv2 = lambda r: (r[2] == r[3][r[0]-1]) ^ (r[2] == r[3][r[1]-1])

print(f"Task 1: {df.apply(iv1, axis=1).sum()}, Task 2: {df.apply(iv2, axis=1).sum()}")

Task 1: 418, Task 2: 616


## **Day 3**

In [96]:
a = pd.read_csv("inputs20/3", header=None)[0].str.split("", expand=True).iloc[:,1:-1].replace({".":0, "#":1}).to_numpy()
f = lambda x, s: sum([a[i, (int(x*i)) % a.shape[1]] for i in range(0, a.shape[0], s)])
print(f"Task 1: {f(3,1)}, Task 2: {f(1,1) * f(3,1) * f(5,1) * f(7,1) * f(.5,2)}")

Task 1: 216, Task 2: 6708199680


## **Day 4**

In [169]:
l = [s.replace("\n", " ") for s in open("inputs20/4").read().split("\n\n")]
nl = []

for s in l:
    tmp = {}
    for kvs in s.split(" "):
        k, v = kvs.split(":")
        tmp[k] = v
    nl.append(tmp)

iv1 = lambda x: (("cid" in x) and len(x) == 8) or (not("cid" in x) and len(x) == 7)
iv2 = lambda x: (
    ("byr" in x) and (1920 <= int(x["byr"]) <= 2002) and
    ("iyr" in x) and (2010 <= int(x["iyr"]) <= 2020) and
    ("eyr" in x) and (2020 <= int(x["eyr"]) <= 2030) and
    ("hgt" in x) and (
        (x["hgt"][-2:] == "cm" and (150 <= int(x["hgt"][:-2]) <= 193)) or
        (x["hgt"][-2:] == "in" and (59 <= int(x["hgt"][:-2]) <= 76))
    ) and
    ("hcl" in x) and len(x["hcl"]) == 7 and (x["hcl"][0] == "#")
        and (set(x["hcl"][1:]) <= {"0","1","2","3","4","5","6","7","8","9","a","b","c","d","e","f"}) and
    ("ecl" in x) and (x["ecl"] in ["amb","blu","brn","gry","grn","hzl","oth"]) and
    ("pid" in x) and len(x["pid"]) == 9 and int(x["pid"]) >= 0 and
    True
)
print(f"Task 1: {sum([iv1(d) for d in nl])}, Task 2: {sum([iv2(d) for d in nl])}")

Task 1: 242, Task 2: 186


## **Day 5**

In [246]:
df = pd.read_csv("inputs20/5", header=None)
df["row"] = df[0].str[:-3].str.replace("F", "0").str.replace("B", "1").apply(lambda x: int(x, 2))
df["col"] = df[0].str[-3:].str.replace("L", "0").str.replace("R", "1").apply(lambda x: int(x, 2))
df["id"] = df["row"] * 8 + df["col"]

t1 = df.sort_values(["row", "col"]).tail(1)["id"].iloc[0]
t2 = (df[df.sort_values("id")["id"].diff() > 1]["id"] - 1).iloc[0]

print(f"Task 1: {t1}, Task 2: {t2}")

Task 1: 901, Task 2: 661


## **Day 6**

In [285]:
l = [x.split("\n") for x in open("inputs20/6").read().split("\n\n")]
t1  = sum([len(set("".join(x))) for x in l])
t2 = sum([len(set.intersection(*[set(x) for x in g])) for g in l])
print(f"Task 1: {t1}, Task 2: {t2}")

Task 1: 6534, Task 2: 3402


## **Day 7**

In [51]:
l = open("inputs20/7").read().split("\n")
bags = [" ".join(s.split(" ")[:2]) for s in l]
M = pd.DataFrame(columns=bags, index=bags, dtype=int).fillna(0)

for s, b in zip(l, bags):
    for c in s.split("contain")[1].split(","):
        if "no" in c: continue
        _, n, *x, _ = c.split(" ")
        M.loc[b, " ".join(x)] = int(n)

M0, S, T = M.copy(), set(), 0
while M.sum().sum() > 0:
    S |= set(M[M.loc[:, "shiny gold"]>0].index)
    T += int(sum(M.loc["shiny gold"]))
    M @= M0
    
print(f"Task 1: {len(S)}, Task 2: {T}")

Task 1: 142, Task 2: 10219


## **Day 8**

In [51]:
# pd.read_csv("inputs20/8t", header=None)
l = open("inputs20/8").read().split("\n"); N = len(l)

for swt in range(-1, N):
    acc, idx, hist = 0, 0, []
    while True:
        if (idx >= N): break
        cmd, val = l[idx].split(" ")
        val = int(val)
        if idx == swt:
            if cmd == "nop": cmd = "jmp"
            elif cmd == "jmp": cmd = "nop"
        if cmd == "nop": idx += 1
        if cmd == "acc": acc += val; idx += 1
        if cmd == "jmp": idx += val
        if idx in hist: break
        hist.append(idx)
    if swt == -1: t1 = acc
    if idx >=  N: t2 = acc

print(f"Task 1: {t1}, Task 2: {t2}")

Task 1: 1337, Task 2: 1358


In [6]:
+2

2